# Voronoi algorithm

## Import and parameters

In [54]:
import queue
import time
import math
import cv2
import matplotlib.pyplot as plt
from skimage.io import imread
import numpy as np
import plotly.express as px
import os
from tifffile import imsave, imwrite
from queue import LifoQueue

## parameters

zstack = 0
NAPlim = 1000 #maximal number of added colored pixels to the original cell
pix_sensitivity = 0 # number of pixels that we cannot distinguish


## paths 
date = 220721
path = 'I:\\biomechanics\\IX83\\220813_MDCK_BEADS_small_conf\\'

list_dir = os.listdir(path)
list_dir

['dz590nm_lmbda561nm_last_frame',
 'dz590nm_lmbda561nm_tl60x12mn',
 'Overview_MDCK_10xPH.vsi',
 '_temp']

In [55]:
index = 1
path_experiment = path + list_dir[index]
phase_images_dir = path_experiment + '\\tif_phase\\CTF_2defocus\\'
segmented_images_dir = path_experiment + '\\tif_segmentation\\'



In [56]:
dirs = os.listdir(phase_images_dir)
phase_image_paths = [] 
for name in dirs :
    if name.endswith('.tif'):
        phase_image_paths.append(phase_images_dir + name)
phase_image_paths

['I:\\biomechanics\\IX83\\220813_MDCK_BEADS_small_conf\\dz590nm_lmbda561nm_tl60x12mn\\tif_phase\\CTF_2defocus\\2defCTF_1180nm+2360nm_30_220814-dz590nm_lmbda561nm_tl60x12mn-001.tif',
 'I:\\biomechanics\\IX83\\220813_MDCK_BEADS_small_conf\\dz590nm_lmbda561nm_tl60x12mn\\tif_phase\\CTF_2defocus\\2defCTF_1180nm+2360nm_30_220814-dz590nm_lmbda561nm_tl60x12mn-002.tif',
 'I:\\biomechanics\\IX83\\220813_MDCK_BEADS_small_conf\\dz590nm_lmbda561nm_tl60x12mn\\tif_phase\\CTF_2defocus\\2defCTF_1180nm+2360nm_30_220814-dz590nm_lmbda561nm_tl60x12mn-003.tif',
 'I:\\biomechanics\\IX83\\220813_MDCK_BEADS_small_conf\\dz590nm_lmbda561nm_tl60x12mn\\tif_phase\\CTF_2defocus\\2defCTF_1180nm+2360nm_30_220814-dz590nm_lmbda561nm_tl60x12mn-004.tif']

In [57]:
dirs = os.listdir(segmented_images_dir)
segmented_image_paths = []
for name in dirs :
    if name.endswith('.tif') and ('median' in name):
        segmented_image_paths.append(segmented_images_dir + name)
segmented_image_paths

['I:\\biomechanics\\IX83\\220813_MDCK_BEADS_small_conf\\dz590nm_lmbda561nm_tl60x12mn\\tif_segmentation\\cp_median_1180nm+2360nm_30_220814-dz590nm_lmbda561nm_tl60x12mn-001.tif',
 'I:\\biomechanics\\IX83\\220813_MDCK_BEADS_small_conf\\dz590nm_lmbda561nm_tl60x12mn\\tif_segmentation\\cp_median_1180nm+2360nm_30_220814-dz590nm_lmbda561nm_tl60x12mn-002.tif',
 'I:\\biomechanics\\IX83\\220813_MDCK_BEADS_small_conf\\dz590nm_lmbda561nm_tl60x12mn\\tif_segmentation\\cp_median_1180nm+2360nm_30_220814-dz590nm_lmbda561nm_tl60x12mn-003.tif']

In [58]:

expanded_image_path = path + 'tif_expanded\\2defCTF_1180nm+2360nm_30_220712-MDCK_FN_FULL_20x_dz590nm-001_'+ str(zstack) + 'zstack.tif' 
output_path = path + 'force_input_220712.txt'

## Expand cells

### Tools 

In [59]:
def flip_vertical(image):
    (a,b)=np.shape(image)
    for i in range(a//2):
        stock=np.copy(image[i])
        image[i]=np.copy(image[a-i-1])
        image[a-i-1]=stock
    return image

In [60]:
def load_image_1stack(path,zstack):
    """load the stack zstack from the path image"""
    segmented_img_zstack = imread(path)    # give a (dz,shape) array with dz=number of stacks in the tiff file
    segmented_img_1stack = segmented_img_zstack[zstack]  # select the stack 'zstack'
    return segmented_img_1stack

In [61]:
def is_in_screen(coordinates,image):
    """input : (x,y) coordinates
               (a,b) shape
       output : boolean """
    (x,y)=coordinates
    (a,b)=np.shape(image)
    return x>=0 and x<a and y>=0 and y<b

In [62]:
def show(image, cmap=None):
    """show images function"""
    image=image/np.max(image)*255
    plt.imshow(image,cmap=cmap)

In [63]:
img_test = load_image_1stack(segmented_image_paths[0],1)
a = img_test[0:None,0:None].shape 
a[0]

974

### Init functions

In [64]:
def init_processed_array(image):
    """return a boolean array from an image with True values for non zeros 
    pixels """
    processed= np.copy(image!=0)
    return processed

def init_queue(image):
    """ initialize a lifo queue,
        the objects inside the queue are quadruplets (coordinates,color,number of added pixels from the original cell),
        filled with non zeros pixels"""
    
    (a,b)=np.shape(image)
    maxsize=(a+1)*(b+1)
    q=queue.Queue(maxsize)

    #fill the queue with the already colored pixels
    for i in range(a):
        for j in range(b):
            if image[i,j]!=0:
                q.put((i,j,image[i,j],0))
    return q


### Voronoi algorithm 1stack

In [65]:
xmin = 0
xmax = -1
ymin = 0
ymax = -1

In [66]:
def expand_1stack_4dir(path,zstack,xmin = 0,xmax = None,ymin = 0, ymax = None):
    """main function of voronoi algorithm"""
    
    segmented_img_1stack = load_image_1stack(path, zstack)
    segmented_img_1stack = segmented_img_1stack[xmin:xmax, ymin:ymax]
    expanded_img_1stack = np.copy(segmented_img_1stack)
    to_be_treated = init_queue(segmented_img_1stack)
    processed = init_processed_array(segmented_img_1stack)
    while (not(to_be_treated.empty())) :
        (cx,cy,color,n)=to_be_treated.get()  # get current coordinates
        for (i,j) in [(0,1),(1,0),(1,2),(2,1)] : 
            if is_in_screen((cx-1+i,cy-1+j),expanded_img_1stack) and not(processed[cx-1+i,cy-1+j]) and n<=NAPlim:
                to_be_treated.put((cx-1+i,cy-1+j,color,n+1))
                expanded_img_1stack[cx-1+i,cy-1+j] = color
                processed[cx-1+i,cy-1+j] = True
    return expanded_img_1stack

In [67]:
def expand_1stack(path,zstack,xmin = 0,xmax = None,ymin = 0, ymax = None):
    """main function of voronoi algorithm"""
    
    segmented_img_1stack = load_image_1stack(path, zstack)
    segmented_img_1stack = segmented_img_1stack[xmin:xmax, ymin:ymax]
    expanded_img_1stack = np.copy(segmented_img_1stack)
    to_be_treated = init_queue(segmented_img_1stack)
    processed = init_processed_array(segmented_img_1stack)
    while (not(to_be_treated.empty())) :
        (cx,cy,color,n)=to_be_treated.get()  # get current coordinates
        for i in range(3):
            for j in range(3):
                if is_in_screen((cx-1+i,cy-1+j),expanded_img_1stack) and not(processed[cx-1+i,cy-1+j]) and n<=NAPlim:
                    to_be_treated.put((cx-1+i,cy-1+j,color,n+1))
                    expanded_img_1stack[cx-1+i,cy-1+j] = color
                    processed[cx-1+i,cy-1+j] = True
    return expanded_img_1stack

### Voronoi algorithm  film

In [68]:
def expand_tl_4dir(path, xmin = 0,xmax = None,ymin = 0, ymax = None):
    segmented_img = imread(path) 
    time = segmented_img.shape[0]
    expanded_img = np.copy(segmented_img)
    for i in range(time):
        print('frame',i+1)
        expanded_img[i] = expand_1stack_4dir(path,i,xmin,xmax,ymin,ymax)
    return expanded_img
    

### Main

In [ ]:
expanded_imgs = []

for i,segmented_image_path in enumerate(segmented_image_paths):   
    print('processing image', i+1,'...')
    expanded_img = expand_tl_4dir(segmented_image_path,zstack)
    expanded_imgs.append(expanded_img)


In [ ]:
expanded_imgs

### Visualisation of first frame 

In [ ]:
for phase_image_path in phase_image_paths:
    phase_img = load_image_1stack(phase_image_path, 0)
    show(phase_img_1stack)

In [ ]:
for segmented_image_path in segmented_image_paths:
    segmented_img = load_image_1stack(segmented_image_path, 0)
    show(segmented_img_1stack)

In [ ]:
for expanded_image_path in expanded_image_paths:
    expanded_img = load_image_1stack(expanded_image_path, 0)
    show(expanded_img_1stack)

In [ ]:
#imwrite(expanded_image_path,expanded_img_1stack)

## Find vertices,edges 

### tools

In [24]:
def is_junction(x,y,image):
    """ determines if (x,y) is a junction in image according to the color of the adjacent pixels looking toward right bottom
    """
    a,b,c,d=image[x,y],image[x,y+1],image[x+1,y],image[x+1,y+1]
    return (((a!=b and a!=c and b!=c) or (a!=b and a!=d and b!=d) or (a!=c and a!=d and d!=c) or (c!=b and d!=c and b!=d)) and a!=0 and b!=0 and c!=0 and d!=0 and (a!=d and b!=c) )

In [25]:
def is_fwj(x,y,image):
    """ determines if (x,y) is a perfect four way junction in image according to the color of the adjacent pixels looking toward right bottom"""
    a,b,c,d=image[x,y],image[x,y+1],image[x+1,y],image[x+1,y+1]
    return a!=b and a!=c and a!=d and b!=c and b!=d and c!=d and a!=0 and b!=0 and c!=0 and d!=0

In [26]:
def find_colors_3(x,y,image):
    """find the three different colors in a 2 by 2 square of first coordinate x,y in image """
    a = image[x,y]
    b = image[x,y+1]
    c = image[x+1,y]
    d = image[x+1,y+1]
    
    if a!=b and a!=c and b!=c:
        return a,b,c
    elif a!=b and a!=d and b!=d:
        return a,b,d
    elif a!=c and a!=d and d!=c:
        return a,c,d
    else :
        return b,c,d

In [27]:
def find_colors_4(x,y,image):
    """find the four different colors in a 2 by 2 square of first coordinate x,y in image """
    a = image[x,y]
    b = image[x,y+1]
    c = image[x+1,y+1]
    d = image[x+1,y]
    
    return a,b,c,d

In [28]:
def find_doubled(c1,c2,c3,c4,c5,c6):
    """two elements are both in {c1,c2,c3} and {c4,c5,c6}
       extract the two doubled element, and the two single elements from among c1,c2,c3,c4,c5,c6 """
    set1={c1,c2,c3}
    set2={c4,c5,c6}
    setDoubles=set1 & set2
    setSingles=set1 ^ set2
    if not setSingles :
        return -1,-1,-1,-1

    else :
        doubled_color_1=setDoubles.pop()
        doubled_color_2=setDoubles.pop()
        single_color_1=setSingles.pop()
        single_color_2=setSingles.pop()

        return doubled_color_1, doubled_color_2, single_color_1, single_color_2

In [29]:
def area(indicesList,vertices):
    x = np.array(vertices.xList)
    y = np.array(vertices.yList)
    indicesList = np.array(indicesList)
    xx = x[indicesList]
    #print(xx)
    sx = np.roll(xx, -1)
    yy = y[indicesList]
    #print(yy)
    sy = np.roll(yy, -1)
    return 0.5*sum(xx * sy -yy * sx)
    

In [30]:
def show_edges(cells_network):
    
    # assign categories
    # use colormap
    colormap = np.array(['r', 'g', 'b'])
    markermap = np.array(['o','x'])
    ecolor = ["b","yellow"]
    
    edge_dict=cells_network.edges.data
    for key in edge_dict :
        edge = edge_dict[key]
        
        i1=edge.bounds[0]
        i2=edge.bounds[1]
        #print(i2)
        x1=cells_network.vertices.vertList[i1].x
        x2=cells_network.vertices.vertList[i2].x
        y1=cells_network.vertices.vertList[i1].y
        y2=cells_network.vertices.vertList[i2].y
        plt.plot((y1,y2),(x1,x2), c = ecolor[edge.extern])
    
def show_vertices(cells_network, size = None):
    colormap = np.array(['r', 'g', 'b'])
    type_categories = cells_network.vertices.get_categories()
    external = cells_network.vertices.get_markers()
    X=np.array(cells_network.vertices.yList)
    Y=np.array(cells_network.vertices.xList)
    
    plt.scatter(X[external == 0],Y[external == 0],c=colormap[type_categories[external == 0]],marker='o', s = size)
    plt.scatter(X[external == 1],Y[external == 1],c=colormap[type_categories[external == 1]],marker='x', s = size)
    

 


In [31]:
def show_cells(cells_network):
    image = np.copy(cells_network.image)
    (a,b) = np.shape(image)
    for i in range(a):
        for j in range(b):
            cell = cells_network.cells.data.get(image[i,j])
            if cell :
                image[i,j]=cell.extern*255
    plt.imshow(image)
    
    

### class definition

In [32]:
class Vertex :
    """one vertex is determined by its coordinates"""
    def __init__(self,n=0,x = -1 ,y = -1,colorList = [], index = -1):
        self.type = n # "0" for three way junctions, "1" for strict for way junctions, "2" for approximated four way junctions
        self.x = x # x coordinate
        self.y = y # y coordinate
        self.extern = 0 # boolean = 1 if external vertex
        self.active = 1 #boolean = 1 if active vertex
        self.colors = colorList
        self.neighbours = set()



class Vertices :
    """list of vertices"""
    
    def __init__(self):
        self.vertList = [] # should be filled with Vertex
        self.xList = [] # should be filled with x coordinates
        self.yList = [] # should be filled with y coordinates
    
    def add_vertex(self,n,x,y,color_list):
        vertex=Vertex(n,x,y,color_list)
        self.vertList.append(vertex)
        self.xList.append(vertex.x)
        self.yList.append(vertex.y)
    
    def get_categories(self):
        """return a colormap array according to the type of junction of each vertex"""
        N = len(self.vertList)
        categories=np.zeros(N, dtype=int)
        for i in range(N):
            categories[i]=self.vertList[i].type
        return categories

    def get_markers(self):
        """return a colormap array according to the type of junction of each vertex"""
        N = len(self.vertList)
        markers=np.zeros(N, dtype=int)
        for i in range(N):
            markers[i]=self.vertList[i].extern
        return markers  

    def nb_active_vertices(self) :
        nb = 0
        for x in self.vertList:
            nb += x.active
        return nb
    
    def nb_INvertices(self):
        """returns the number of inner vertices in the list"""
        n = 0
        for x in self.vertList :
            n += (1-x.extern)
        return n
    
    def nb_EXvertices(self):
        """returns the number of external vertices in the list"""
        n = 0
        for x in self.vertList :
            n += x.active*x.extern
        return n
    
    def nb_fwj(self):
        n = 0
        for x in self.vertList :
            if x.type and not(x.extern):
                n +=1
        return n

In [33]:
        
class Edge :
    """one edge is defined by the indices of its boundaries"""
    def __init__(self):
        self.bounds = []
        self.extern = 0 #boolean = 1 if edge is external
                
class EdgeDict :
    """dictionnary of Edges"""
    def __init__(self):
        self.data={} #storing the edges by their unique identifier which is a duet containing the colors separated by the given edge
        
        
    def getEdge(self,c1,c2):
        """ectracts the Edge delimiting c1 and c2 from data dict if it exists, create one otherwise"""
        edge=self.data.get((c1,c2))
        if edge :
            return edge
        else :
            newEdge=Edge()
            return newEdge
    
    def updateEdge(self,edge,c1,c2):
        """updates or add the edge in the dict"""
        self.data.update({(c1,c2) : edge})

    def nb_INedge(self):
        """returns the number of inner edge in the dictionary"""
        n = 0
        for key in self.data :
            n += (1-self.data[key].extern)
        return n
    
    def nb_EXedge(self):
        """returns the number of external edge in the dictionary"""
        n = 0
        for key in self.data :
            n += self.data[key].extern
        return n
    

In [34]:
class Cell :
    """a cell is defined by its number of vertices and the list of its vertices"""
    def __init__(self,nbv=0):
        self.nbv = nbv #number of vertices of a cell, must be equal to the length of the following list
        self.verticesList = [] #list of indices of vertices delimiting the cell
        self.extern = 0 #boolean = 1 if cell is external
        self.neighbours = set()
        
    def add_vertex(self, vertex_index):
        self.nbv+=1
        self.verticesList.append(vertex_index)
    


class CellsDict:
    """Dictionary of the cells of an image, the key of a cell is its color"""
    def __init__(self,nb_colors):
        self.data = {} # Dict of Cell objects
        
    def getCell(self,color):
        """extracts the cell color from the data dictionnary"""
        cell=self.data.get(color)
        if cell :
            return cell
        else :
            newCell=Cell()
            return newCell
    
    def nb_INcell(self):
        """returns the number of inner cells in the dictionary"""
        n = 0
        for key in self.data :
            n += (1-self.data[key].extern)
        return n
    
    def nb_EXcell(self):
        """returns the number of external cells in the dictionary"""
        n = 0
        for key in self.data :
            n += self.data[key].extern
        return n
    
    def updateCell(self,cell, color):
        """updates or adds the cell in the dict"""
        self.data.update({color : cell})
        
    def add_index2cell(self, color,vertex_index):
        """adds the index of a vertex to the corresponding cell in the data dictionary"""
        cell = self.getCell(color)
        cell.add_vertex(vertex_index)
        self.updateCell(cell,color)
    
    def add_indices2cell(self,color_list,i):
        for c in color_list:
            self.add_index2cell(c,i)

    

In [53]:
    
        
class Voronoi :
    """Voronoi diagram with direct access to its vertices, edges, and cells properties"""
    def __init__(self,expanded_image):
        self.image = expanded_image
        nb_color = np.max(expanded_image)
        self.vertices = Vertices() #filled with Vertex objects
        self.edges = EdgeDict()
        self.cells = CellsDict(nb_color)
    
    

    def add_index2edges_2(self,c1,c2,i):
        """adds the index i to one boundary of the edges between c1 and c2, and update neighbours sets if needed"""
        (c1,c2) = (min(c1,c2),max(c1,c2))
        edge = self.edges.getEdge(c1,c2)
        n = len(edge.bounds)

        for j in range(n):
            self.vertices.vertList[edge.bounds[j]].neighbours.add(i)
            self.vertices.vertList[i].neighbours.add(edge.bounds[j])
        edge.bounds.append(i)
        self.edges.updateEdge(edge,c1,c2)

    
    def add_index2edges_3(self,c1,c2,c3,i):
        """adds the index i to one boundary of the edges between c1 and c2, c2 and c3, c3 and c1 of the edges dictionnary, and updates neighbours if needed"""
        self.add_index2edges_2(c1,c2,i)
        self.add_index2edges_2(c2,c3,i)
        self.add_index2edges_2(c1,c3,i)
    
    def add_edges_fwj(self,cul,cur,cbl,cbr,i):
        """adds four edges for four way junctions, and updates neighbours if needed"""
        self.add_index2edges_2(cul,cur,i)
        self.add_index2edges_2(cur,cbl,i)
        self.add_index2edges_2(cbl,cbr,i)
        self.add_index2edges_2(cbr,cul,i)

    def add_edges_fwj_approx(self, doubled_color_1,doubled_color_2,single_color_1, single_color_2,i) :
        """adds four edges for approximated four way junctions and updates neighbours if needed"""
        self.add_index2edges_2(doubled_color_1,single_color_1,i)
        self.add_index2edges_2(doubled_color_2,single_color_1,i)
        self.add_index2edges_2(doubled_color_1,single_color_2,i)
        self.add_index2edges_2(doubled_color_2,single_color_2,i)
    
    def add_vertex(self,n,x,y,color_list):
        """adds the vertex with parameters n,x,y to the vertices list of self"""
        self.vertices.add_vertex(n,x,y,color_list)
    
    def add_indices2cell(self,color_list,i):
        """adds the vertex i to the cell of color c"""
        self.cells.add_indices2cell(color_list,i)
        
    def remove_cell(self,color):
        """marks the vertices of the cell as external vertices and removes the cell from the cell dictionary"""
        cell = self.cells.data.get(color)
        if cell:
            for index_vertex in cell.verticesList:
                vertex = self.vertices.vertList[index_vertex]
                vertex.active = 0
                vertex.extern = 1
            for nc in cell.neighbours :
                self.cells.data[nc].extern = 1
                self.cells.data[nc].neighbours.remove(color)
            self.cells.data.pop(color)
            
    def remove_edge(self,key):
        """remove the edge defined by key from the dictionary of edges"""
        edge = self.edges.data.pop(key)
            
    def nb_ext_vertices(self,color) :
        """counts and returns the number of external vertices of the cell"""
        cell = self.cells.data.get(color)
        nb = 0
        for i in cell.verticesList :
            nb += (1 - self.vertices.vertList[i].active)
        return nb

    def nb_non_ext_vertices(self,color) :
        """counts and returns the number of NON external vertices of the cell"""
        cell = self.cells.data.get(color)
        nb = 0
        for i in cell.verticesList :
            nb += self.vertices.vertList[i].active
        return nb

    def find_invaders(self,edge) :
        n = len(edge.bounds)
        vcolors = []
        intersect = set()
        for i in range(n):
            vcolors.append(set(self.vertices.vertList[edge.bounds[i]].colors))
#             intersect = intersect & vcolors[i]
#         c1 = intersect.pop()
#         c2 = intersect.pop()
        
        for i in range(n):
            for j in range(i+1,n) :
                if vcolors[i] == vcolors[j] :
                    return (i,j)
        return (-1,-1)
        

    def clean_edges(self) : 
        edges = self.edges.data
        for key in edges :
            edge = edges.get(key)
            n = len(edge.bounds)
            if n > 2 :
                print(n)
                (i1,i2) = self.find_invaders(edge)
                if (i1,i2) == (-1,-1):
                    raise Exception("not good")
                else :
                    for i in range(n) :
                        if i!=i1 and i!=i2 :
                            self.vertices.vertList[edge.bounds[i]].neighbours.remove(edge.bounds[i1])
                            self.vertices.vertList[edge.bounds[i]].neighbours.remove(edge.bounds[i2])
                    if i1<i2 :
                        edge.bounds.pop(i2)
                        edge.bounds.pop(i1)
                    else :
                        edge.bounds.pop(i1)
                        edge.bounds.pop(i2)
                        
            
                


    def find_vertices_edges(self,pix_sensitivity) :
        """ input : - pix_sensitivity : integer, number of pixels to consider four way junctions
            modify the 'junctions' and 'edges' attribute of self  """

        # initialization

        (a,b) = np.shape(self.image)
        spotted_junctions = np.zeros((a,b))
        index = 0
        
        # browse the image
        for x in range(a-1):
            for y in range(b-1):
                twj = 0
                if is_junction(x,y,self.image) and not(spotted_junctions[x,y]):
                    twj = 1 # boolean : True if and only if it is a three way junction
                    fwj = 0 # boolean : True if and only if it is a strict four way junction
                    fwj_approx = 0 # boolean : True if and only if it is an approximated four way junction
                    c1,c2,c3 = find_colors_3(x,y,self.image)
                    
                    if is_fwj(x,y,self.image):
                        #print(x,y)
                        fwj = 1
                        twj = 0
                        cul, cur, cbl, cbr = find_colors_4(x,y,self.image) #gives colors in the upper/bottom left/right corners
                        
                    for i in range(min(pix_sensitivity,a-1-x)):
                        for j in range(min(pix_sensitivity,b-1-y)):
                            if is_junction(x+i,y+j,self.image):
                                if i+j != 0 : # maj of fwj only if i!=0 or j!=0
                                    fwj_approx = 1 
                                    fwj = 0
                                    twj = 0
                                    spotted_junctions[x+i,y+j] = 1
                                    c4,c5,c6 = find_colors_3(x+i,y+j,self.image)
                    
                    # filling the output
                    if fwj_approx:
                        doubled_color_1, doubled_color_2, single_color_1, single_color_2 = find_doubled(c1,c2,c3,c4,c5,c6)
                        if (doubled_color_1, doubled_color_2, single_color_1, single_color_2)!=(-1,-1,-1,-1):
                            #print(doubled_color_1, doubled_color_2, single_color_1, single_color_2)
                            self.add_vertex(2,x,y,[doubled_color_1, doubled_color_2, single_color_1, single_color_2])
                            self.add_edges_fwj_approx(doubled_color_1,doubled_color_2,single_color_1, single_color_2,index)
                            
                        else :
                            print(x,y)
                            twj = 1
                    
                    if fwj :
                        self.add_vertex(1,x,y,[cul,cur,cbl,cbr])
                        self.add_edges_fwj(cul,cur,cbl,cbr,index)
                        #self.add_indices2cell([cul,cur,cbl,cbr],index)
                    
                    if twj:
                        self.add_vertex(0,x,y,[c1,c2,c3])
                        self.add_index2edges_3(c1,c2,c3,index)
                        #self.add_indices2cell([c1,c2,c3],index)
                    spotted_junctions[x,y] = 1
                    index += 1
        self.clean_edges()
   
    def order_cell(self,color):
        """orders the vertices of the cell color so that in the vertices list of the cell, v_i and v_i+1 are connected by an edge"""
        
        #finding one bounds to be the seed (useful only in the case of an extern cell)
        
        cell = self.cells.data[color]
        vert_set = set(cell.verticesList)
        v = vert_set.pop()
        ordered_vertlist = [v]
        while (vert_set) :
            intersect = vert_set & self.vertices.vertList[v].neighbours
            if intersect :
                v = intersect.pop()
                vert_set.remove(v)
                ordered_vertlist.append(v)
            else :
               break
            
        #ordering the vertices
        
        vert_set = set(cell.verticesList)
        vert_set.remove(v)
        ordered_vertlist = [v]
        while (vert_set) :
            #print('vert_set : ',vert_set, '\nneighbours : ', self.vertices.vertList[v].neighbours,'\n')
            intersect = vert_set & self.vertices.vertList[v].neighbours
            v = intersect.pop()
            vert_set.remove(v)
            ordered_vertlist.append(v)
        
        #making anti clockwise if clockwise
        
        if area(ordered_vertlist,self.vertices) >= 0 :
            ordered_vertlist = ordered_vertlist[::-1]
        
        #updating the vertices list
        
        cell.verticesList = ordered_vertlist
        

    def order_cells(self):
        """orders the vertices list of all cells of self"""
        for color in self.cells.data :
            self.order_cell(color)
            
    def find_cells_neighbours(self):
        cells = self.cells
        edges = self.edges
        for c1, c2 in edges.data :
            cell1 = cells.data.get(c1)
            cell2 = cells.data.get(c2)
            if cell1 and cell2 :
                cells.data[c1].neighbours.add(c2)
                cells.data[c2].neighbours.add(c1)
            
    
    def find_cells(self, processed):
        """updates the cell dictionary of self by browsing the vertices list and keeps only the bigest connex component"""
        nbv = len(self.vertices.vertList)
        to_be_treated = LifoQueue(maxsize = nbv)
        to_be_treated.put(0)
        while(not(to_be_treated.empty())):
            cv = to_be_treated.get()
            if not(processed[cv]) :
                processed[cv] = 1
                for vi in self.vertices.vertList[cv].neighbours :
                    to_be_treated.put(vi)
                for colors in self.vertices.vertList[cv].colors :
                    self.add_indices2cell([colors],cv)
        
        if np.sum(processed) <= len(processed)/2 :
            nbv = len(self.vertices.vertList)
            for color in self.cells.data :
                self.cells.data[color].verticesList = []
            self.find_cells(processed)
        else :
            for i in range(nbv) :
                if not(processed[i]):
                    self.vertices.vertList[i].extern = 1
                    self.vertices.vertList[i].active = 0
        self.find_cells_neighbours()
        self.order_cells()

    

                
    def clean_cell_bounds(self):
        """removes the external cells to fit the right boundary conditions according to [1]"""
        (a,b)=np.shape(self.image)
        to_remove = set()
        for x in [0,a-1]:
            for y in range(b):
                color = self.image[x,y]
                to_remove.add(color)
        for y in [0,b-1]:
            for x in range(a):
                color = self.image[x,y]
                to_remove.add(color)
        for c in to_remove :
            self.remove_cell(c)
        
        b = 1
        while (b == 1) :
            b = 0
            to_remove = set()
            for color,cell in self.cells.data.items() :
                if all(self.cells.data[nc].extern == 1 for nc in cell.neighbours) or any((self.vertices.vertList[v].type == 1 and self.vertices.vertList[v].extern == 1) for v in cell.verticesList):
                    b = 1
                    to_remove.add(color)
            for color in to_remove : 
                self.remove_cell(color)
    
        
    def clean_edge_bounds(self):
        """removes the useless edges and verticies"""
        key_to_remove=[]
        for edge_key in self.edges.data :
            edge = self.edges.data[edge_key]
            n = len(edge.bounds)
            if n !=2 :
                key_to_remove.append(edge_key)
            else :
                edge = self.edges.data.get(edge_key)
                v1 = self.vertices.vertList[edge.bounds[0]]
                v2 = self.vertices.vertList[edge.bounds[1]]
                if v1.extern and v2.extern :
                    key_to_remove.append(edge_key)
                elif v1.extern or v2.extern :
                    edge.extern = 1
                    v1.active = 1
                    v2.active = 1
        for key in key_to_remove :
            self.remove_edge(key)
        
    def clean_indices(self):
        """re index all vertices to have indices from 0 to nb_vertices after cleaning boundaries"""
        
        correspondancy = {} # dictionary : key = old index, object = new index
        nb_vert_tot = len(self.vertices.vertList)
        new_index = 1
        index_to_pop = []
        for i in range(nb_vert_tot):
            if self.vertices.vertList[i].active :
                correspondancy.update({ i : new_index })
                new_index += 1
            else :
                index_to_pop.append(i)
        
        
        for key in self.edges.data :
            edge = self.edges.data[key]
            edge.bounds[0] = correspondancy[edge.bounds[0]] - 1
            edge.bounds[1] = correspondancy[edge.bounds[1]] - 1
        
        for key in self.cells.data :
            cell = self.cells.data[key]
            n = len(cell.verticesList)
            new_indices=[]
            for k in range(n) :
                new_index = correspondancy.get(cell.verticesList[k])
                if new_index :
                    new_indices.append(new_index-1)
            cell.verticesList = new_indices
            cell.nbv = len(new_indices)
        n=len(index_to_pop)
        for i in range(n) :
            self.vertices.vertList.pop(index_to_pop[n-i-1])
            self.vertices.xList.pop(index_to_pop[n-i-1])
            self.vertices.yList.pop(index_to_pop[n-i-1])
                

    def bounds(self) :
        image=self.image

        self.clean_cell_bounds()
        self.clean_edge_bounds()
        self.clean_indices()

        

            

In [ ]:
#fig = plt.figure(figsize = (8,8))

#plt.imshow(segmented_img_1stack[820:840,75:100] ,cmap="gray")

fig = plt.figure(figsize = (8,8))
plt.imshow(expanded_img_1stack[1030:1080,2080:2120] ,cmap="gray")
fig = plt.figure(figsize = (8,8))
plt.imshow(segmented_img_1stack[1000:1200,2000:2200] , cmap = "ocean")
fig = plt.figure(figsize = (8,8))
plt.imshow(phase_img_1stack[1080:1100,2600:2650] , cmap = "gray")

print(segmented_img_1stack[1090,2621])
print(segmented_img_1stack[1090,2615])
print(segmented_img_1stack[1085,2615])

# Output writing

In [51]:
def export_data(path,cells_network,i,t) :
    
    print ("experiment number :", str(i) ,".........")
    print("time :",str(t))
    
    ## OPENING FILE and INIT VARIABLES
    output = open(path, 'w')
    
    C_NUM = len(cells_network.cells.data)
    IN_CNUM = cells_network.cells.nb_INcell()
    EX_CNUM = cells_network.cells.nb_EXcell()
    
    E_NUM = len(cells_network.edges.data)
    IN_E_NUM = cells_network.edges.nb_INedge()
    EX_E_NUM = cells_network.edges.nb_EXedge()
    
    V_NUM = cells_network.vertices.nb_active_vertices()
    IN_V_NUM = cells_network.vertices.nb_INvertices()
    EX_V_NUM = cells_network.vertices.nb_EXvertices()
    
    fwj = cells_network.vertices.nb_fwj()
    
    fwj_eq = 3*(IN_V_NUM + fwj)/2 - fwj
    
    print("internal v : " , IN_V_NUM ,'\ne : ',IN_E_NUM, '\nN : ', IN_CNUM, '\n' )
    print("vertices : ", V_NUM)
    print("four way junctions : ",fwj, '\nfour way equation : ', fwj_eq)

    ## HEADER

    # cells related numbers
    output.write("### C_NUM " + str(C_NUM) + "\n")
    output.write("### IN_CNUM " + str(IN_CNUM) + "\n")
    output.write("### EX_CNUM " + str(EX_CNUM) + "\n")
    
    # edges related numbers
    output.write("### E_NUM " + str(E_NUM) + "\n")
    output.write("### IN_E_NUM " + str(IN_E_NUM) + "\n")
    output.write("### EX_E_NUM " + str(EX_E_NUM) + "\n")
    
    # vertices related numbers
    output.write("### V_NUM " + str(V_NUM) + "\n")
    output.write("### IN_V_NUM " + str(IN_V_NUM) + "\n")
    output.write("### EX_V_NUM " + str(EX_V_NUM) + "\n")
    
    ## CONTENT
    
    # vertices
    for i in range(V_NUM) :
        output.write("V[" + str(i) + "] " + str(cells_network.vertices.yList[i]) + " " + str(cells_network.vertices.xList[i]))
        if cells_network.vertices.vertList[i].extern :
            output.write(" Ext")
        output.write ('\n')
    output.write('\n')
        
    # edges
    for i, key in enumerate(cells_network.edges.data):
        output.write("E[" + str(i) + "] " + str(cells_network.edges.data[key].bounds[0]) + " " + str(cells_network.edges.data[key].bounds[1]))
        if cells_network.edges.data[key].extern :
            output.write(" Ext")
        output.write ('\n')
    output.write('\n')
        
    # cells
    for i, key in enumerate(cells_network.cells.data):
        output.write("C[" + str(i) + "] " + str(len(cells_network.cells.data[key].verticesList)) + " :" )
        for x in cells_network.cells.data[key].verticesList :
            output.write(" " + str(x))
        if cells_network.cells.data[key].extern :
            output.write("  Ext")
        output.write ('\n')
    

In [ ]:
def count_between(voronoi,xmin,xmax,ymin,yamx):
    e = 0
    for key in voronoi.edges.data :
        e += (1-voronoi.edges.data[key].extern)
    
    v = 0
    for x in voronoi.vertices.vertList :
        if x.x<xmax and x.x>xmin and x.y<ymax and x.y>ymin :
            v += (1-x.extern)
    
    n = 0
    for key in voronoi.cells.data :
        n += (1-voronoi.cells.data[key].extern)

    
    return e,v,n



## main

In [ ]:
start = time.time()
cells_networks = []

output_dir = path_experiment + '//force_input'
os.makedirs(output_dir, exist_ok=True)

for i,expanded_img in enumerate(expanded_imgs[0:]) :
    nb_frame = expanded_img.shape[0]
    for t in range(nb_frame):
        print(t)
        cn = Voronoi(expanded_img[t])
        cn.find_vertices_edges(pix_sensitivity)
        cn.find_cells(np.zeros(len(cn.vertices.vertList)))
        cn.bounds()
        cells_networks.append(cn)
        output_path =os.path.join(output_dir,str(date)+ '_' +str(i+1) + '-' + str(t+1) + '.txt') 
        export_data(output_path, cn,i,t)
    
end = time.time()

print("Time : ")
print(end-start)
# cn.find_edges(pix_sensitivity)

In [ ]:
fig = plt.figure(figsize = (10,10))

#show(test, cmap="gray")
#show(segmented_img_1stack)
show(phase_img_1stack,cmap='gray')
#show(expanded_img_1stack,cmap='gray')
#show_cells(cn)
show_vertices(cn, 5)
show_edges(cn)

v = cn.vertices.vertList[0]
plt.scatter(v.y,v.x)
#plt.savefig(path + "vertices.png")

